In [2]:
import pandas as pd

In [29]:
# load data
data = pd.read_excel('./data/HR Employee Attrition.xlsx')

data.head()

,AGE,AGE.1,BusinessTravel,HourlyRate,OverTime,ClassAttrition,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Junior Senior,35,Travel_Rarely,30,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Junior Senior,37,Travel_Rarely,30,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Junior Senior,35,Travel_Rarely,30,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Senior,51,Travel_Frequently,30,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Senior,54,Non-Travel,30,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AGE             1470 non-null   object 
 1   AGE.1           1470 non-null   int64  
 2   BusinessTravel  1470 non-null   object 
 3   HourlyRate      1470 non-null   int64  
 4   OverTime        1470 non-null   object 
 5   ClassAttrition  1470 non-null   object 
 6   Unnamed: 6      0 non-null      float64
 7   Unnamed: 7      4 non-null      object 
 8   Unnamed: 8      11 non-null     object 
 9   Unnamed: 9      9 non-null      object 
 10  Unnamed: 10     11 non-null     object 
 11  Unnamed: 11     9 non-null      object 
 12  Unnamed: 12     5 non-null      object 
 13  Unnamed: 13     2 non-null      object 
dtypes: float64(1), int64(2), object(11)
memory usage: 160.9+ KB


In [31]:
# detele unnmaed column
for col in data.columns:
    if 'Unnamed' in col:
        del data[col]

In [32]:
data.head()

,AGE,AGE.1,BusinessTravel,HourlyRate,OverTime,ClassAttrition
0,Junior Senior,35,Travel_Rarely,30,Yes,No
1,Junior Senior,37,Travel_Rarely,30,No,No
2,Junior Senior,35,Travel_Rarely,30,No,No
3,Senior,51,Travel_Frequently,30,No,No
4,Senior,54,Non-Travel,30,Yes,No


In [33]:
# knowledge age
def fuzzy_age_knowldege(age, category):
    if category == 'JUNIOR':
        if age <= 22 or age >= 32:
            return 0
        elif age > 22 and age <= 30:
            return (age - 22) / (30 - 22)
        elif age > 30 and age <= 32:
            return (32 - age) / (32 - 30)
    elif category == 'JUNIOR SENIOR':
        if age <= 30 or age >= 40:
            return 0
        elif age > 30 and age <= 38:
            return (age - 30) / (38 - 30)
        elif age > 38 and age <= 40:
            return (40 - age) / (40 - 38)
    elif category == 'SENIOR':
        if age <= 38 or age >= 60:
            return 0
        elif age > 38 and age <= 60:
            return (age - 38) / (60 - 38)
        

In [34]:
def fuzzy_businessTravel_knowledge(travel_frequency, category):
    if category == 'NON':
        if travel_frequency <= 2 or travel_frequency >= 5:
            return 0
        elif travel_frequency > 2 and travel_frequency <= 5:
            return (travel_frequency - 2) / (5 - 2)
    elif category == 'RARELY':
        if travel_frequency <= 3 or travel_frequency >= 8:
            return 0
        elif travel_frequency > 3 and travel_frequency <= 8:
            return (travel_frequency - 3) / (8 - 3)
    elif category == 'FREQUENTLY':
        if travel_frequency <= 6 or travel_frequency >= 10:
            return 0
        elif travel_frequency > 6 and travel_frequency <= 10:
            return (travel_frequency - 6) / (10 - 6)

In [35]:
def fuzzy_hourlyRate_knowledge(hourly_rate, category):
    if category == 'LOW':
        if hourly_rate <= 30 or hourly_rate >= 60:
            return 0
        elif hourly_rate > 30 and hourly_rate <= 50:
            return (50 - hourly_rate) / (50 - 30)
    elif category == 'HIGH':
        if hourly_rate <= 50 or hourly_rate >= 80:
            return 0
        elif hourly_rate > 50 and hourly_rate <= 80:
            return (hourly_rate - 50) / (80 - 50)

In [36]:
def fuzzy_overtime_knowledge(overtime_hours, category):
    if category == 'NO':
        if overtime_hours <= 5 or overtime_hours >= 8:
            return 0
        elif overtime_hours > 5 and overtime_hours <= 8:
            return (overtime_hours - 5) / (8 - 5)
    elif category == 'YES':
        if overtime_hours <= 6 or overtime_hours >= 10:
            return 0
        elif overtime_hours > 6 and overtime_hours <= 10:
            return (overtime_hours - 6) / (10 - 6)

In [37]:
# Fuzzy inference rules
def fuzzy_inference_rule(age, business_travel, hourly_rate, overtime):
    # Rule 1
    rule1 = min(fuzzy_age_knowldege(age, 'JUNIOR'), fuzzy_businessTravel_knowledge(business_travel, 'NON'))
    # Rule 2
    rule2 = min(fuzzy_businessTravel_knowledge(age, 'SENIOR'), fuzzy_businessTravel_knowledge(business_travel, 'FREQUENTLY'))
    # Rule 3
    rule3 = min(fuzzy_hourlyRate_knowledge(hourly_rate, 'HIGH'), fuzzy_overtime_knowledge(overtime, 'NO'))
    # Rule 4
    rule4 = min(fuzzy_overtime_knowledge(hourly_rate, 'LOW'), fuzzy_overtime_knowledge(overtime, 'YES'))

    # Aggregation
    aggregated_membership = max(rule1, rule2, rule3, rule4)

    return aggregated_membership

In [38]:
# Defuzzification
def defuzzification(membership_values):
    return sum(membership_values) / len(membership_values)

## Simulasi

In [39]:
# Misalkan Input data
input_age = 37
input_business_travel = 3
input_hourly_rate = 32
input_overtime = 6

In [42]:
# Fuzzyfication
membership_age_junior = fuzzy_age_knowldege(input_age, 'JUNIOR')
membership_age_senior = fuzzy_age_knowldege(input_age, 'SENIOR')
membership_business_travel_non = fuzzy_businessTravel_knowledge(input_business_travel, 'NON')
membership_business_travel_rarely = fuzzy_businessTravel_knowledge(input_business_travel, 'RARELY')
membership_business_travel_frequently = fuzzy_businessTravel_knowledge(input_business_travel, 'FREQUENTLY')
membership_hourly_rate_low = fuzzy_hourlyRate_knowledge(input_hourly_rate, 'LOW')
membership_hourly_rate_high = fuzzy_hourlyRate_knowledge(input_hourly_rate, 'HIGH')
membership_overtime_no = fuzzy_overtime_knowledge(input_overtime, 'NO')
membership_overtime_yes = fuzzy_overtime_knowledge(input_overtime, 'YES')


In [43]:
# Inference
rule1 = min(membership_age_junior, membership_business_travel_non)
rule2 = min(membership_age_senior, membership_business_travel_frequently)
rule3 = min(membership_hourly_rate_high, membership_overtime_no)
rule4 = min(membership_hourly_rate_low, membership_overtime_yes)


In [44]:
# Aggregation
aggregated_membership = max(rule1, rule2, rule3, rule4)

# Defuzzification
output = defuzzification([aggregated_membership])

# Output
print("Fuzzy Output (Class Attrition):", output)


Fuzzy Output (Class Attrition): 0.0
